# Here we show that the muon weighted field average gets the same results when calculated in the grid method and the moments method

## 1) The Moments Method
### Basic setup

In [1]:
import sys
import numpy as np
import pandas as pd

basedir='/Users/bono/Desktop/gm2FieldAnalysis/MuonConvolution/'

#import the relevant field moments functions
path = basedir + 'field_info'
sys.path.insert(0,path)
from format_field import field_team_to_standard_moments as ftts
from field_grid import *

#import the relevant synthesis tools
path = basedir + 'synthesis'
sys.path.insert(0,path)
from spacial_tools import moments_method as mm

path= basedir + 'tracker_info'
sys.path.append(path)
from beam_moments import *
from transform_beam import *
from beam_moments import get_normalized_distribution_moments as gndm

### Get the field moments
Load the field/odb pickle file for the 60 hour dataset

In [2]:
path = basedir + 'synthesis/data/60H_field_ctag_dqm.pkl'
df = pd.read_pickle(path)

get the ctag weighted field moments and convert to standard representation

In [3]:
total_ctags = df["ctags"].sum()
b = [
    (df["D"]*df["ctags"]).sum()/total_ctags,
    (df["NQ"]*df["ctags"]).sum()/total_ctags,
    (df["SQ"]*df["ctags"]).sum()/total_ctags,
    (df["NS"]*df["ctags"]).sum()/total_ctags
    ]
c,s = ftts(b)

### Get the beam moments

get the dataframe representing tracker 12

In [4]:
df_tracker = get_df_tracker()
mask12 = (df_tracker["Station"] == 'station12')
df_12 = df_tracker[mask12].copy()
df_12['counts_norm'] = df_12['counts']/df_12['counts'].sum()
df_12.head()

,index,radial,vertical,counts,Station,counts_norm
0,2,-59.0,-59.0,0.0,station12,0.0
1,3,-59.0,-57.0,0.0,station12,0.0
2,4,-59.0,-55.0,0.0,station12,0.0
3,5,-59.0,-53.0,0.0,station12,0.0
4,6,-59.0,-51.0,0.0,station12,0.0


get the normalized moments beam moments

In [5]:
x = df_12['radial']
y = df_12['vertical']
weights = df_12['counts']
N = 3
I_norm, J_norm = gndm(N,x,y,weights)

### Display all the moments and their contributions

In [6]:
print(c,'\n')
print(s,'\n')
print(I_norm,'\n')
print(J_norm,'\n')
print(c*I_norm,'\n')
print(s*J_norm,'\n')

[ 8.41413353e+02 -9.48434999e-03 -1.18420911e+00] 

[0.         0.23283668 0.        ] 

[1.         0.15097058 0.10640059] 

[ 0.         -0.00012891  0.00056273] 

[ 8.41413353e+02 -1.43185781e-03 -1.26000550e-01] 

[ 0.00000000e+00 -3.00142634e-05  0.00000000e+00] 



### and get the "moments method" result

In [7]:
moments_result = mm(c,s,I_norm,J_norm)
moments_result

841.2858909405439

## 2) The Grid Method
### convert the bloch field moments to a field grid

In [8]:
df_field_grid = get_field_grid(b)
fig = plot_field(df_field_grid)

### and having left the beam-grid in it's native form, get the grid result

In [9]:
grid_result = (df_12['counts']*df_field_grid['B']).sum()/(df_12['counts'].sum())
grid_result

841.285890940544

In [10]:
grid_result - moments_result

1.1368683772161603e-13

__we see that the difference in the two methods is tiny__

